<a href="https://colab.research.google.com/github/IamMarking/Tensorflow/blob/master/3-2%20MNIST%20%E7%AE%80%E5%8D%95%E5%88%86%E7%B1%BB%E7%89%88%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensorflow 手写数字集合学习笔记

[神经网络学习](https://www.zybuluo.com/hanbingtao/note/433855)

In [3]:
import tensorflow as tf

old_v = tf.logging.get_verbosity()

tf.logging.set_verbosity(tf.logging.ERROR)

from tensorflow.examples.tutorials.mnist import input_data  # 导入手写数字集合

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  # 加载数据

tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


调试代码：
- batch_size = 100 可以修改
- 添加隐藏层
- 激活函数
- 初始化的值
- 损失函数交叉熵
- 优化器学习率的调整
- 训练次数

In [4]:
batch_size = 100  # 每次放入 100 张图片进行训练，每张图片维度是 784 维

n_batch = mnist.train.num_examples // batch_size  # 计算一共有多少个批次

# 定义两个 placeholder 容器
x = tf.placeholder(tf.float32,[None,784])   
y = tf.placeholder(tf.float32,[None,10])

# 构建一个简单的神经网络
# W = tf.Variable(tf.zeros([784,10]))
W = tf.Variable(tf.truncated_normal([784,10],stddev=0.1))  # 初始化权重矩阵：截短正态分布
b = tf.Variable(tf.zeros([10]))

prediction = tf.nn.softmax(tf.matmul(x,W)+b)  # 100 * 784 * 784 * 10 = 100 * 10 * 10 * 1 = 100 * 1

loss = tf.reduce_mean(tf.square(y-prediction))  # mse 
# loss = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction)  # 交叉熵损失

train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放在一个布尔型变量中
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))  # argmax 返回一维张量中最大的值所在的位置

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  # 转换类型，由布尔型到 float32

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(20):  # 训练 20 次
        for batch in range(n_batch):  # 训练的批次数
          
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)  # 每 100 张依次进行训练
            
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})  # 运行计算图，并喂入数据
            
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})  # 每一次的准确率变化
        print('Iter ' + str(epoch) + ', Testing Accuracy ' + str(acc) )


Iter 0, Testing Accuracy 0.5662
Iter 1, Testing Accuracy 0.7449
Iter 2, Testing Accuracy 0.7807
Iter 3, Testing Accuracy 0.8179
Iter 4, Testing Accuracy 0.8483
Iter 5, Testing Accuracy 0.8625
Iter 6, Testing Accuracy 0.8703
Iter 7, Testing Accuracy 0.875
Iter 8, Testing Accuracy 0.8787
Iter 9, Testing Accuracy 0.8825
Iter 10, Testing Accuracy 0.8856
Iter 11, Testing Accuracy 0.8886
Iter 12, Testing Accuracy 0.891
Iter 13, Testing Accuracy 0.8929
Iter 14, Testing Accuracy 0.8957
Iter 15, Testing Accuracy 0.8964
Iter 16, Testing Accuracy 0.8975
Iter 17, Testing Accuracy 0.8992
Iter 18, Testing Accuracy 0.8995
Iter 19, Testing Accuracy 0.901


In [0]:
batch_size = 100  # 1 次放入 100 张图片进行训练，每张图片维度是 784 维

n_batch = mnist.train.num_examples // batch_size  # 批次的个数

# 创建 3 个 placeholder 容器
x = tf.placeholder(tf.float32,[None,784])   
y = tf.placeholder(tf.float32,[None,10])
keep_prob = tf.placeholder(tf.float32)

# 构建一个简单的神经网络
# W = tf.Variable(tf.zeros([784,10]))
W1 = tf.Variable(tf.truncated_normal([784,200],stddev=0.1))  # 初始化权重
b1 = tf.Variable(tf.zeros([200])+0.1)
L1 = tf.nn.tanh(tf.matmul(x,W1)+b1)
L1_drop = tf.nn.dropout(L1,keep_prob)

W2 = tf.Variable(tf.truncated_normal([200,200],stddev=0.1))
b2 = tf.Variable(tf.zeros([200])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop,W2)+b2)
L2_drop = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.truncated_normal([200,100],stddev=0.1))
b3 = tf.Variable(tf.zeros([100])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop,W3)+b3)
L3_drop = tf.nn.dropout(L3,keep_prob)


W4 = tf.Variable(tf.truncated_normal([100,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)

prediction = tf.nn.softmax(tf.matmul(L3_drop,W4)+b4)  # 100 * 784 * 784 * 10 = 100 * 10 * 10 * 1 = 100 * 1

# loss = tf.reduce_mean(tf.square(y-prediction))  # mse 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))  # 交叉熵损失

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

init = tf.global_variables_initializer()

# 结果存放在一个布尔型变量中
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))  # argmax 返回一维张量中最大的值所在的位置

# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  # 转换类型，由布尔型到 float32

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(20):  # 训练 21 次
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)  # 每 100 张依次进行训练
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})  # 每一次的准确率变化
        print('Iter ' + str(epoch) + ', Testing Accuracy ' + str(test_acc) + ', Training Accuracy ' + str(train_acc))

Iter 0, Testing Accuracy 0.9076, Training Accuracy 0.9051818
Iter 1, Testing Accuracy 0.9264, Training Accuracy 0.9243636
Iter 2, Testing Accuracy 0.9347, Training Accuracy 0.9338727
Iter 3, Testing Accuracy 0.94, Training Accuracy 0.93954545
Iter 4, Testing Accuracy 0.9442, Training Accuracy 0.9436727
Iter 5, Testing Accuracy 0.9472, Training Accuracy 0.94969094
Iter 6, Testing Accuracy 0.9511, Training Accuracy 0.95223635
Iter 7, Testing Accuracy 0.9506, Training Accuracy 0.95336366
Iter 8, Testing Accuracy 0.9564, Training Accuracy 0.9580909
Iter 9, Testing Accuracy 0.956, Training Accuracy 0.9590545
Iter 10, Testing Accuracy 0.957, Training Accuracy 0.9608909
Iter 11, Testing Accuracy 0.9582, Training Accuracy 0.96294546
Iter 12, Testing Accuracy 0.9603, Training Accuracy 0.96410906
Iter 13, Testing Accuracy 0.9614, Training Accuracy 0.9644
Iter 14, Testing Accuracy 0.9619, Training Accuracy 0.9662182
Iter 15, Testing Accuracy 0.9634, Training Accuracy 0.9675636
Iter 16, Testing Ac

**没有过拟合**

Iter 18, Testing Accuracy 0.9661, Training Accuracy 0.97007275

Iter 19, Testing Accuracy 0.9657, Training Accuracy 0.9714182

**过拟合**

Iter 18, Testing Accuracy 0.9743, Training Accuracy 0.9912909

Iter 19, Testing Accuracy 0.9755, Training Accuracy 0.9917091